# ID2222 Homework 3: Mining Data Streams

## Hongyi Luo (hluo@kth.se) & Yage Hao (yage@kth.se)

### 0. Preprocessing

dataset: https://snap.stanford.edu/data/web-Stanford.html 

***Dataset statistics***  
Nodes	281903  
Edges	2312497  
Nodes in largest WCC	255265 (0.906)  
Edges in largest WCC	2234572 (0.966)  
Nodes in largest SCC	150532 (0.534)  
Edges in largest SCC	1576314 (0.682)  
Average clustering coefficient	0.5976  
Number of triangles	11329473  
Fraction of closed triangles	0.002889  
Diameter (longest shortest path)	674  
90-percentile effective diameter	9.7  

In [54]:
import numpy as np
import random

In [76]:
# import the graph dataset
dataset = set()
with open("web-Stanford.txt") as f:
    for i in range(4, 10000+4): # discard desciptions and pick first 10000 elements to ease the computation
        content = next(f).strip().split()
        if content[0] != content[1]:
            dataset.add((content[0], content[1]))
print(f'dataset contains {len(dataset)} edges')

dataset contains 10000 edges


### 1. TRIEST-BASE

In [97]:
def TRIEST_BASE(M, dataset=dataset):
    """
    This function estimates number of global triangles by TRIEST_BASE algorithm.
    
    argument: 
        M: size of reservoir sample, M>=6
        dataset, =dataset defaultly.
    return:
        number of global triangle count.
    """
    S = set() # edge sample from reservoir sampling
    t = 0 # time
    tau = 0 # global counter for the estimation of the global number of triangles

    
    def sample_edge(edge, t, S, tau, M=M):
        """
        This function is the reservoir sampling process.
        Notice that each edge item in the sample has equal probability.
    
        argument:
            edge: an arbitrary edge.
            t: time instance.
            S: edge sample.
            M: size of reservoir sample.
            tau: global counter.
        return:
            True or False: whether the input edge will replace an existing edge in the edge sample.
            tau: if updated.
        """
        # if t<=M, the edge on the stream at time t 
        # is deterministically inserted in S
        if t <= M:
            return [True]
    
        # if t>M, TRIEST-BASE flips a biased coin with heads probability M/t
        # If the outcome is heads, it chooses an existing edge in S uniformly at random,
        # remove it and insert the current edge on time t into S 
        elif random.random() <= (M/t): 
            del_edge = random.sample(S, 1)[0]
            S.remove(del_edge)
            #print(1,tau)
            tau = update_counters('-', del_edge, S, tau) 
            #print(2,tau)
            return [True, tau]
    
        # Otherwise S is not modified
        return [False]
    
    
    def update_counters(operation, edge, S, tau):
        """
        This function compute neighborhood of vertices and update global and local counters.
    
        argument:
            operation: '+' or '-', means insert or delete respectively.
            edge: an arbitrary edge.
            S: edge sample.
            tau: global counter.    
        return:
            tau: updated global counter.
        """
        u = edge[0]
        v = edge[1]
    
        # construct neighborhood of u and v respectively
        # $ N^S_u = {v in V^(t): (u,v) in S} $
        u_neighbor = set()
        v_neighbor = set()
        for vertices_pair in S:
            # neighborhood of u
            if u == vertices_pair[0]:
                u_neighbor.add(vertices_pair[1])
            elif u == vertices_pair[1]:
                u_neighbor.add(vertices_pair[0])
            # neighborhood of v
            if v == vertices_pair[0]:
                v_neighbor.add(vertices_pair[1])
            elif v == vertices_pair[1]:
                v_neighbor.add(vertices_pair[0])
        
        # construct shared neighborhood of u and v
        # $ N^S_u,v = intersection(N^S_u, N^S_v) $
        shared_neighbor = set.intersection(u_neighbor, v_neighbor)
        #if shared_neighbor != set():
        #   print(operation, len(shared_neighbor))
    
        # update counters
        for c in (u_neighbor & v_neighbor):
            if operation == '+':
                tau += 1
                tau_u = len(u_neighbor) + 1 # local counter for a subset of the nodes u in V^(t)
                tau_v = len(v_neighbor) + 1 # local counter for a subset of the nodes v in U^(t)
                tau_c = len(shared_neighbor) + 1 # local counter for a subset of the nodes in shared neighborhood of u and v
            elif operation == '-':
                tau -= 1
                tau_u = len(u_neighbor) - 1
                if tau_u <= 0:
                    del tau_u
                tau_v = len(v_neighbor) - 1
                if tau_v <= 0:
                    del tau_v
                tau_c = len(shared_neighbor) - 1
                if tau_c <= 0:
                    del tau_c
        return tau
    
    
    for edge in dataset:
        #if t%1000 == 0:
        #    print(f'element index = {t}, tau = {tau}')
            
        t += 1 # update time
        result = sample_edge(edge, t, S, tau)
        if result[0] is True:
            S.add(edge)
            if len(result) > 1:
                tau = result[1] # update tau value of deletion step
            tau = update_counters('+', edge, S, tau)  
        
    epsilon = (t*(t-1)*(t-2)) / (M*(M-1)*(M-2))
    if epsilon < 1:
        epsilon = 1
    return epsilon*tau

#### Test 1.1

We set M=5000 in the above sampling case and remember the size of the dataset we use is 10000.  
We can see that when t<=M, our estimate number of triangles by TRIEST_BASE algorithm is exactly the same as true value.

In [95]:
print('Compute true value:')
true_value = TRIEST_BASE(len(dataset)) # M = len(dataset) = 10000
print(f'True value is {true_value}.')

print('\nCompute estimate value by reservoir sampling:')
estimate_value = TRIEST_BASE(5000) # pick arbitrary M=5000
print(f'Estimate value is {estimate_value}.')

print(f'\nError: {abs(estimate_value - true_value)} triangles.')
print(f'Error rate: {abs(estimate_value - true_value)/true_value}.')

Compute true value:
element index = 0, tau = 0
element index = 1000, tau = 37
element index = 2000, tau = 296
element index = 3000, tau = 1002
element index = 4000, tau = 2367
element index = 5000, tau = 4555
element index = 6000, tau = 7557
element index = 7000, tau = 11411
element index = 8000, tau = 16712
element index = 9000, tau = 22707
True value is 30190.0.

Compute estimate value by reservoir sampling:
element index = 0, tau = 0
element index = 1000, tau = 37
element index = 2000, tau = 296
element index = 3000, tau = 1002
element index = 4000, tau = 2367
element index = 5000, tau = 4555
element index = 6000, tau = 4499
element index = 7000, tau = 4435
element index = 8000, tau = 4475
element index = 9000, tau = 4602
Estimate value is 38043.41416566627.

Error: 7853.414165666269 triangles.
Error rate: 0.2601329634205455.


#### Test 1.2

With multiple runs, we hypothesis that the results (estimation of number of global triangles) distributed normally. And if the sample mean is biased compared to the true value, we suppose that the algorithm is somehow biased.

In [98]:
estimate_list = []
for i in range(20):
    print(i)
    estimate_value = TRIEST_BASE(5000) # pick arbitrary M=5000
    estimate_list.append(estimate_value)
    
mean_value = np.mean(estimate_list)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [102]:
print(f'run TRIEST_BASE for 20 times with M=5000, result list: \n{estimate_list}')
print(f'\nmean value is {mean_value}')

run TRIEST_BASE for 20 times with M=5000, result list: 
[35994.79951980792, 38411.52460984394, 41004.30252100841, 38107.43337334934, 40796.24009603842, 35786.73709483794, 38643.59423769508, 37131.140456182475, 38267.48139255703, 38891.66866746699, 36042.81392557023, 39291.7887154862, 41188.357743097244, 35594.67947178872, 34530.360144057624, 38947.685474189675, 38195.45978391357, 37459.23889555823, 37731.32052821129, 38827.64945978392]

mean value is 38042.21380552221


#### Test 1.3

The estimation of global number of triangles is very sensative to the choice of M.
As M tends to the length of the original dataset, the estimation tends to the true value and the variance of results in multiple runs shrinks.

In [104]:
print(f'true value = {true_value}')
for M in [1000, 3000, 5000, 7000, 9000]:
    estimate_value = TRIEST_BASE(M)
    print(f'M = {M}, estimate number of global triangles = {estimate_value}')

true value = 30190.0
M = 1000, estimate number of global triangles = 61165.07411218834
M = 3000, estimate number of global triangles = 37655.9891144502
M = 5000, estimate number of global triangles = 36290.88835534214
M = 7000, estimate number of global triangles = 33878.99204053664
M = 9000, estimate number of global triangles = 30470.151635973678


### 2. TRIEST-IMPR

In [106]:
def TRIEST_IMPR(M, dataset=dataset):
    """
    This function estimates number of global triangles by TRIEST_IMPR algorithm.
    
    argument: 
        M: size of reservoir sample, M>=6
        dataset, =dataset defaultly.
    return:
        number of global triangle count.
    """
    S = set() # edge sample from reservoir sampling
    t = 0 # time
    tau = 0 # global counter for the estimation of the global number of triangles

    
    def sample_edge(edge, t, S, M=M):
        """
        This function is the reservoir sampling process.
        Notice that each edge item in the sample has equal probability.
    
        argument:
            edge: an arbitrary edge.
            t: time instance.
            S: edge sample.
            M: size of reservoir sample.
        return:
            True or False: whether the input edge will replace an existing edge in the edge sample.
        """
        # if t<=M, the edge on the stream at time t 
        # is deterministically inserted in S
        if t <= M:
            return True
    
        # if t>M, TRIEST-BASE flips a biased coin with heads probability M/t
        # If the outcome is heads, it chooses an existing edge in S uniformly at random,
        # remove it and insert the current edge on time t into S 
        elif random.random() <= (M/t): 
            del_edge = random.sample(S, 1)[0]
            S.remove(del_edge)
            # TRIEST-IMPR never decrements the counters when an edge is removed from S
            return True
    
        # Otherwise S is not modified
        return False
    
    
    def update_counters(edge, S, tau, t, M=M):
        """
        This function compute neighborhood of vertices and update global and local counters.
    
        argument:
            edge: an arbitrary edge.
            S: edge sample.
            tau: global counter.  
            t: time instance.
            M: sample size, default =M.
        return:
            tau: updated global counter.
        """
        u = edge[0]
        v = edge[1]
    
        # construct neighborhood of u and v respectively
        # $ N^S_u = {v in V^(t): (u,v) in S} $
        u_neighbor = set()
        v_neighbor = set()
        for vertices_pair in S:
            # neighborhood of u
            if u == vertices_pair[0]:
                u_neighbor.add(vertices_pair[1])
            elif u == vertices_pair[1]:
                u_neighbor.add(vertices_pair[0])
            # neighborhood of v
            if v == vertices_pair[0]:
                v_neighbor.add(vertices_pair[1])
            elif v == vertices_pair[1]:
                v_neighbor.add(vertices_pair[0])
        
        # construct shared neighborhood of u and v
        # $ N^S_u,v = intersection(N^S_u, N^S_v) $
        shared_neighbor = set.intersection(u_neighbor, v_neighbor)
        #if shared_neighbor != set():
        #    print(len(shared_neighbor))
    
        # update counters
        # all the calls to update_counters have operation '+'
        # preform a weighted increase using weight
        # $ eta^t = max{1, (t-1)(t-2)/(M(M-1))} $
        eta = ((t-1)*(t-2)) / (M*(M-1))
        if eta < 1:
            eta = 1
            
        for c in (u_neighbor & v_neighbor):
            tau += eta
            tau_u = len(u_neighbor) + eta # local counter for a subset of the nodes u in V^(t)
            tau_v = len(v_neighbor) + eta # local counter for a subset of the nodes v in U^(t)
            tau_c = len(shared_neighbor) + eta # local counter for a subset of the nodes in shared neighborhood of u and v
        return tau
    
    
    for edge in dataset:
        #if t%1000 == 0:
        #    print(f'element index = {t}, tau = {tau}')
            
        t += 1 # update time
        tau = update_counters(edge, S, tau, t) # call update_counters unconditionally
        
        if sample_edge(edge, t, S) == True:
            S.add(edge)
        
    return tau

#### Test 2.1

Similar to test 1.1, we first set M=5000 in the above improved sampling case and the size of the dataset we use is 10000.  
We can see that when t<=M, our estimate number of triangles by TRIEST_IMPR algorithm is exactly the same as true value.

In [94]:
print('Compute true value:')
true_value = TRIEST_IMPR(len(dataset)) # M = len(dataset) = 10000
print(f'True value is {true_value}.')

print('\nCompute estimate value by reservoir sampling:')
estimate_value = TRIEST_IMPR(5000) # pick arbitrary M=4000
print(f'Estimate value is {estimate_value}.')

print(f'\nError: {abs(estimate_value - true_value)} triangles.')
print(f'Error rate: {abs(estimate_value - true_value)/true_value}.')

Compute true value:
element index = 0, tau = 0
element index = 1000, tau = 37
element index = 2000, tau = 296
element index = 3000, tau = 1002
element index = 4000, tau = 2367
element index = 5000, tau = 4555
element index = 6000, tau = 7557
element index = 7000, tau = 11411
element index = 8000, tau = 16712
element index = 9000, tau = 22707
True value is 30190.

Compute estimate value by reservoir sampling:
element index = 0, tau = 0
element index = 1000, tau = 37
element index = 2000, tau = 296
element index = 3000, tau = 1002
element index = 4000, tau = 2367
element index = 5000, tau = 4555
element index = 6000, tau = 7610.927455811263
element index = 7000, tau = 11942.63614002814
element index = 8000, tau = 18017.876664293035
element index = 9000, tau = 25427.016077535725
Estimate value is 35295.12952070433.

Error: 5105.1295207043295 triangles.
Error rate: 0.16910001724757634.


#### Test 2.2

We run 20 times of TRIEST_IMPR algorithm with M=5000.  
Comparing results with the ones obtained by TRIEST_BASE algorithm, we can see that the variance of improved algorithm is lower and the mean of the improved algorithm is closer to the true value as we expected.

In [107]:
estimate_list_impr = []
for i in range(20):
    print(i)
    estimate_value = TRIEST_IMPR(5000) # pick arbitrary M=5000
    estimate_list_impr.append(estimate_value)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [113]:
mean_base = np.mean(estimate_list)
mean_impr = np.mean(estimate_list_impr)
var_base = np.var(estimate_list)
var_impr = np.var(estimate_list_impr)

print(f'true value is {true_value}')
print(f'mean from TRIEST_BASE is {mean_base}')
print(f'mean from TRIEST_IMPR is {mean_impr}')
print(f'variance from TRIEST_BASE is {var_base}')
print(f'variance from TRIEST_IMPR is {var_impr}')

true value is 30190.0
mean from TRIEST_BASE is 38042.21380552221
mean from TRIEST_IMPR is 35072.02691547922
variance from TRIEST_BASE is 3170998.664030402
variance from TRIEST_IMPR is 423135.8354098646
